In [2]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math

In [25]:
monthly=pd.read_csv('./output_data/p_and_e_monthly.csv').set_index('MonthId')

In [26]:
monthly

,Flow,Exceedance,Year,Month,WaterYear,WaterMonth,EWRChannelA,EWRChannelC,EWRChannelD,EWRChannelE,...,PowerTurbine4,Power,PerfMwPerCumec,Power_max,Power_min,Turbines_max,Turbines_min,Energy,Days,CapFactor
MonthId,,,,,,,,,,,,,,,,,,,,,
1924.10,116.046674,0.998677,1924.0,10.0,1924.0,1.0,28.193548,4.322581,2.322581,2.000000,...,NaN,8.794568,0.073948,14.935851,0.000000,1,0,6543.158689,31,0.048859
1924.11,153.527057,0.991633,1924.0,11.0,1924.0,2.0,23.000000,4.000000,2.000000,2.000000,...,NaN,21.906859,0.141356,30.764268,15.824578,1,1,15772.938368,30,0.121705
1924.12,275.311093,0.831355,1924.0,12.0,1924.0,3.0,26.000000,4.000000,3.000000,2.000000,...,NaN,47.619043,0.171599,80.774741,29.113645,2,1,35428.567672,31,0.264550
1925.01,691.530484,0.458516,1925.0,1.0,1924.0,4.0,37.129032,5.677419,4.000000,2.677419,...,44.479896,117.132660,0.170994,189.157311,73.295899,4,2,87146.699053,31,0.650737
1925.02,1193.453064,0.278393,1925.0,2.0,1924.0,5.0,49.000000,8.000000,7.000000,5.000000,...,45.233110,180.932441,0.152284,188.262949,174.271686,4,4,121586.600682,28,1.005180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017.05,2194.251076,0.156387,2017.0,5.0,2016.0,8.0,55.000000,13.000000,13.000000,17.000000,...,39.663234,158.652936,0.074586,175.077861,143.051018,4,4,118037.784680,31,0.881405
2017.06,1117.935234,0.312067,2017.0,6.0,2016.0,9.0,53.000000,12.000000,12.000000,13.000000,...,44.199018,154.903353,0.140692,186.152604,95.374353,4,2,111530.414134,30,0.860574
2017.07,482.543823,0.575129,2017.0,7.0,2016.0,10.0,50.000000,10.000000,9.000000,9.000000,...,NaN,62.698034,0.128339,94.719590,42.214410,2,1,46647.337499,31,0.348322


In [13]:
data=pd.read_csv('./old/power_energy.csv')
data['Date']=pd.to_datetime(data['Date'],format="%d/%m/%Y")
data=data.set_index(pd.DatetimeIndex(data['Date']))
data=data.drop(['Date'],axis=1)

data['Year']=data.index.year
data['Month']=data.index.month
data['Day']=data.index.day

data['MonthId']=data['Year']+data['Month']/100
data['WaterYear']=data.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
data['WaterMonth']=data.apply((lambda x: (x['Month']-9 if x['Month']>=10 else x['Month']+3)),axis=1)
data['mmPerf']=data['Total_Power']/data['Inflow']

In [35]:
mm_monthly=data[['MonthId','mmPerf']].groupby("MonthId").mean()
mm_monthly['mmFlow']=data[['MonthId','Inflow']].groupby("MonthId").mean()

In [36]:
mm_monthly

,mmPerf,mmFlow
MonthId,,
1924.10,0.075056,116.383871
1924.11,0.142480,153.979000
1924.12,0.171784,275.676774
1925.01,0.170757,692.740645
1925.02,0.153302,1193.681786
...,...,...
2017.05,0.076024,2195.379677
2017.06,0.141730,1118.893667
2017.07,0.127777,482.787742


In [38]:
monthly['mmPerf']=mm_monthly['mmPerf']
monthly['mmFlow']=mm_monthly['mmFlow']

monthly['PerfRatio']=monthly['mmPerf']/monthly['PerfMwPerCumec']
monthly['FlowRatio']=monthly['mmFlow']/monthly['Flow']

In [52]:
out=monthly[['mmPerf','Flow','PerfMwPerCumec','PerfRatio','FlowRatio']]

In [53]:
out.to_csv('./old/ratios.csv')

In [34]:
monthly['PerfRatio'].mean()

0.9966885833914851

In [40]:
monthly['FlowRatio'].mean()

1.00060200336066

In [48]:
daily=pd.read_csv('./output_data/ngonye_flow_daily.csv')
daily['Date']=pd.to_datetime(daily['Date'],format="%Y-%m-%d")
daily=daily.set_index(pd.DatetimeIndex(daily['Date']))
daily=daily.drop(['Date'],axis=1)

In [45]:
daily

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,WaterMonth,WaterDay,Volume
Date,,,,,,,,,,,,,
1924-10-01,1924-10-12,100.0,1.111331,111.133124,0.999,1924,10,1,1924.10,1924,1,1,0.009602
1924-10-02,1924-10-13,100.0,1.111331,111.133124,0.999,1924,10,2,1924.10,1924,1,2,0.009602
1924-10-03,1924-10-14,100.0,1.111331,111.133124,0.999,1924,10,3,1924.10,1924,1,3,0.009602
1924-10-04,1924-10-15,100.0,1.111331,111.133124,0.999,1924,10,4,1924.10,1924,1,4,0.009602
1924-10-05,1924-10-16,100.0,1.111331,111.133124,0.999,1924,10,5,1924.10,1924,1,5,0.009602
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-26,2017-10-07,204.0,0.995184,203.017496,0.950,2017,9,26,2017.09,2016,12,361,0.017541
2017-09-27,2017-10-08,204.0,0.995184,203.017496,0.950,2017,9,27,2017.09,2016,12,362,0.017541
2017-09-28,2017-10-09,204.0,0.995184,203.017496,0.950,2017,9,28,2017.09,2016,12,363,0.017541


In [49]:
daily['FlowRation']=daily['Flow']/data['Inflow']

In [51]:
daily['FlowRation'].mean()

0.9994086614026201